In [2]:
"""ATE: Automatic Term Extraction
"""

import pandas as pd

import ginza # pip install -U ginza ja_ginza
import nltk  # pip install nltk
import spacy # pip install nltk
import pke   # pip install git+https://github.com/boudinfl/pke.git

In [6]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')

In [7]:
df[df['Published_Year']==2008][['Title','Vol', 'No']].head()

,Title,Vol,No
223,適応ファジィ推論ニューラルネットワークを用いた視覚障害者用学習型位置情報提供システム,46,1
224,記述言語を用いた細胞生理学実験シミュレーション手法,46,1
225,混合脂質膜の透過光スペクトル特性とその自己相関関数に基づく味識別,46,1
226,筋芽細胞の分化と細胞膜電位の変化,46,1
227,生体歩行計測に基づく足首駆動歩行シミュレーション,46,1


In [8]:
years = df["Published_Year"].unique()
years

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [9]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()
    
titles = docs[2022]
print("\n".join(titles[:10]))

生体医工学ウェブ辞典（第一分冊）
市販・試作ウエアラブルデバイスのフィールド適用事例の紹介と課題抽出
飲料摂取をともなうデスクワーク中の生理反応から見た遠隔生体計測の課題
ウエアラブルデバイスによる心拍と呼吸データの比較事例
日常・産業場面におけるウエアラブルデバイス利活用のための問題意識
NIRSを用いた認知介入による認知機能の改善効果研究の最前線
近赤外分光法を用いた脳機能計測アプリケーションの展開
近赤外分光計測を用いたハイパースキャンデータの解析
機能性近赤外分光装置のためのソフトウェア開発の現状と将来展望
拡散相関分光法による組織血流イメージング：原理と応用


In [10]:
def get_key_phrase(spacy_model, text, n = 5, stopwords=None):
    """キーフレーズ処理

    Args:
        text ([type]): キーフレーズ対象の文字列
        n (int, optional): [description]. Defaults to 10.
    Returns:
        [type]: [description]
    """
    #MultipartiteRankなどのキーフレーズの手法については以下のサイトが参考になります
    #https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part5.html
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text, language='ja', normalization=None, stoplist=stopwords)
    extractor.candidate_selection( pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
    extractor.candidate_weighting(threshold=0.10, method='average', alpha=1.1)
    key_phrase = extractor.get_n_best(n)
    
    return  key_phrase

In [22]:
from spacy.lang.ja import stop_words
pke.lang.stopwords['ja_ginza'] = 'ja'
spacy_model = spacy.load("ja_ginza")  # GiNZAモデルの読み込み
stopwords = list(stop_words.STOP_WORDS)
stopwords.extend(['検討','研究','開発','展望','事例','課題','試作','提案']) 
print(stopwords)
nltk.corpus.stopwords.words = stopwords

['ば', 'つけ', 'おら', 'ほか', 'から', 'ら', 'および', 'あり', 'に', 'にて', 'とき', 'きっかけ', 'ん', 'いわ', 'すぐ', 'ため', 'でき', 'ね', 'せい', 'いい', 'まま', 'つい', 'の', 'だ', 'なく', 'つつ', 'つ', 'ちゃん', 'はじめ', 'かつて', 'え', 'らしい', 'ま', 'す', 'どう', 'られる', 'あ', 'そして', 'それ', 'なお', 'うち', 'そこ', 'ただし', 'もの', 'さ', 'それぞれ', 'しまう', 'やっ', 'い', 'しか', 'る', 'くる', 'そう', 'ご', 'まで', 'たち', 'と', 'いう', 'た', 'ながら', 'よく', 'すべて', 'へ', 'ほど', 'な', 'なり', 'ず', 'か', 'たり', 'かつ', 'ほとんど', 'しよう', 'こと', 'ます', 'は', 'もと', 'ぶり', 'できる', 'し', 'なっ', 'よれ', 'のみ', '一', 'ぬ', 'ところ', 'が', 'あるいは', 'で', 'いる', 'のち', 'ある', 'こ', 'よっ', 'あまり', 'とっ', 'です', 'など', 'ない', 'たら', 'もっ', 'かけ', 'よ', 'あっ', 'おけ', 'ここ', 'する', 'さん', 'べき', 'れる', 'いっ', 'せる', 'くん', 'き', 'しまっ', 'さらに', 'かなり', 'より', 'また', 'ごと', 'おい', 'だけ', 'これ', 'いずれ', 'だっ', 'れ', 'ほぼ', 'いく', 'おり', 'なし', 'なかっ', 'よう', 'なけれ', 'よる', 'せ', 'こう', 'いつ', 'み', 'を', 'ひと', 'たい', 'この', 'られ', 'しかし', 'なる', 'あれ', 'や', 'ち', 'もう', 'て', 'も', 'その', 'お', 'なら', 'とも', '検討', '研究', '開発', '展望', '事例', '課題', '試作', '提案']


In [23]:
text = "。\n".join(docs[2022][:10])
print(text)
get_key_phrase(spacy_model, text, n=150, stopwords=stopwords)

生体医工学ウェブ辞典（第一分冊）。
市販・試作ウエアラブルデバイスのフィールド適用事例の紹介と課題抽出。
飲料摂取をともなうデスクワーク中の生理反応から見た遠隔生体計測の課題。
ウエアラブルデバイスによる心拍と呼吸データの比較事例。
日常・産業場面におけるウエアラブルデバイス利活用のための問題意識。
NIRSを用いた認知介入による認知機能の改善効果研究の最前線。
近赤外分光法を用いた脳機能計測アプリケーションの展開。
近赤外分光計測を用いたハイパースキャンデータの解析。
機能性近赤外分光装置のためのソフトウェア開発の現状と将来展望。
拡散相関分光法による組織血流イメージング：原理と応用


[('ウエアラブル デバイス 利活用', 0.12535370369887078),
 ('認知 介入', 0.12195927604415156),
 ('認知 機能', 0.11989406339398705),
 ('問題 意識', 0.11432452298690556),
 ('産業 場面', 0.10740202803665067),
 ('遠隔 生体 計測', 0.08765361070320592),
 ('呼吸 データ', 0.08211644637787247),
 ('最前線', 0.08169748487306891),
 ('生理 反応', 0.07847261021733803),
 ('ハイパー スキャン データ', 0.046598353249644384),
 ('組織 血流 イメージング', 0.034527900418305)]

In [24]:
text = "。\n".join(docs[2019][:200])
get_key_phrase(spacy_model, text, n=50, stopwords=stopwords)

[('妥当性', 0.00781554420833879),
 ('人工知能', 0.006919144170376882),
 ('取り組み', 0.006180078205723661),
 ('システム', 0.005981315810498641),
 ('アプローチ', 0.005463716780686997),
 ('デバイス', 0.005330774967191458),
 ('プロデューサー', 0.005308708291478731),
 ('有効性', 0.00524162378397937),
 ('教育 環境 改善', 0.005215344437731576),
 ('インターフェイス 操作 能力', 0.005157516951021665),
 ('ヒト 身体', 0.005034908921173622),
 ('心理的', 0.0050292153119129305),
 ('心拍 変動 特性', 0.004994746253105341),
 ('ノイズ', 0.0049562614443356455),
 ('集中 学習', 0.004945986246439209),
 ('血管 形成 術中', 0.004934196628964714),
 ('機能的 mri データ', 0.00485179876267092),
 ('特徴 抽出', 0.004839049136903719),
 ('血液 浄化 療法', 0.004831333110405897),
 ('homing', 0.0047548087710929535),
 ('塑性 変形 モニター', 0.004664806672317486),
 ('構造的', 0.0046090812021004355),
 ('シャント 雑音 分析', 0.0045791578024663784),
 ('vpg', 0.004570416388276),
 ('メディカル', 0.004562964958293424),
 ('全脳 モデリング', 0.004558260351997574),
 ('versajet', 0.00455771815832728),
 ('超音波 プローブ', 0.004546079149387671),
 ('マルチ 剛体 リンク モデル